In [1]:
# Translated to .py by Anna Fernández
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:37:04 2018


import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable

df = pd.read_csv("optimaDataset.dat",'\t')
database = db.Database("optimaDataset",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)

### The condition CostCarCHF was not originally there.
exclude =  ((Choice == -1) + (  CostCarCHF   <  0  )) >0
database.remove(exclude)

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_CAR	 = Beta('ASC_CAR',0,None,None,0)
ASC_SM	 = Beta('ASC_SM',0,None,None,0)
ASC_PT	 = Beta('ASC_PT',0,None,None,1)
BETA_COST	 = Beta('BETA_COST',0,None,None,0)
BETA_DIST_CAR	 = Beta('BETA_DIST_CAR',0,None,None,0)
BETA_DIST_SM	 = Beta('BETA_DIST_SM',0,None,None,0)
BETA_TIME_CAR	 = Beta('BETA_TIME_CAR',0,None,None,0)
BETA_TIME_PT	 = Beta('BETA_TIME_PT',0,None,None,0)

# Define here arithmetic expressions for name that are not directly 
# available from the data


#Utilities
CAR = ASC_CAR + BETA_TIME_CAR * TimeCar + BETA_COST * CostCarCHF + BETA_DIST_CAR * distance_km
SM = ASC_SM + BETA_DIST_SM * distance_km
PT = ASC_PT + BETA_TIME_PT * TimePT + BETA_COST * MarginalCostPT
V = {1: CAR,2: SM,0: PT}
av = {1: 1,2: 1,0: 1}

# Logit model, with availability conditions
logprob = models.loglogit(V,av,Choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_specific_optima"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")


                Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_CAR         0.661    0.102    6.48 9.08e-11         0.112         5.88   
ASC_SM          0.142    0.175   0.807    0.419          0.31        0.456   
BETA_COST     -0.0682  0.00729   -9.37        0        0.0115        -5.95   
BETA_DIST_CAR  0.0098  0.00383    2.56   0.0104       0.00578         1.69   
BETA_DIST_SM   -0.239   0.0205   -11.7        0        0.0538        -4.45   
BETA_TIME_CAR -0.0451  0.00588   -7.68  1.6e-14         0.011        -4.12   
BETA_TIME_PT  -0.0133   0.0016   -8.34        0       0.00297        -4.49   

               Rob. p-value  
ASC_CAR            4.16e-09  
ASC_SM                0.648  
BETA_COST          2.66e-09  
BETA_DIST_CAR        0.0903  
BETA_DIST_SM       8.61e-06  
BETA_TIME_CAR      3.79e-05  
BETA_TIME_PT       7.28e-06  
Nbr of observations: 1906
LL(0) =    -2093.955
LL(beta) = -1242.694
rho bar square = 0.403
Output file: logit_specific_optima~00.html
